# Relatório 2

**Nome:** Thiago Lopes <br>
**Matrícula:** 20100358 <br>
**Turma:** T2

# Bibliotecas python utilizadas

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable
import sympy

# Funçoes Auxiliares

In [ ]:
def pivot(A, b, k):
    max_index = np.argmax(np.abs(A[k:, k])) + k
    
    if max_index != k:
        A[[k, max_index]] = A[[max_index, k]]
        b[[k, max_index]] = b[[max_index, k]]


In [ ]:
def back_substitution(U, y):
    m = len(y)
    x = np.zeros(m)
    
    for i in range(m - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
        
    return x


In [ ]:
def forward_substitution(L, b, Pivot):
    m = len(b)
    y = np.zeros(m)
    
    for i in range(m):
        y[i] = b[Pivot[i] - 1] - np.dot(L[i, :i], y[:i])        
    return y


# Métodos implementados (Gaussian Elimination, LU Decomposition):

In [ ]:
def gauss_elimination(A, b, use_pivoting=False):
    n = len(b)
    A = A.astype(float)
    b = b.astype(float)

    #eliminacao com ou sem pivotamento
    for k in range(n-1):
        if use_pivoting:  
            pivot(A, b, k)

        for i in range(k+1, n):
            m = A[i][k] / A[k][k]
            A[i][k] = 0
            
            for j in range(k+1, n):
                A[i][j] -= m * A[k][j]
                
            b[i] -= m * b[k]

    return back_substitution(A, b)


In [ ]:
def LU_decomposition(A, b):
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    m, n = A.shape
    
    if m != n:
        raise ValueError("Matrix must be square")
    
    Pivot = np.arange(1, m + 1)
    PdU = 1.0
    Info = 0
    LU = A.copy()

    for j in range(n):
        pivot(LU, b, j)
        
        Pivot[j], Pivot[np.argmax(np.abs(LU[j:, j])) + j] = Pivot[np.argmax(np.abs(LU[j:, j])) + j], Pivot[j]

        if LU[j, j] == 0:
            if Info == 0:
                Info = j + 1
            continue

        PdU *= LU[j, j]

        if abs(LU[j, j]) != 0:
            r = 1 / LU[j, j]
            for i in range(j + 1, m):
                Mult = LU[i, j] * r
                LU[i, j] = Mult
                LU[i, j + 1:] -= Mult * LU[j, j + 1:]
        else:
            if Info == 0:
                Info = j + 1

    if Info != 0:
        raise ValueError(f"Matrix is singular at row {Info}")

    y = forward_substitution(LU, b, Pivot)

    x = back_substitution(LU, y)

    return x, LU, Pivot, PdU, Info


# Exercícios

## Exercício 1

In [ ]:
# Gauss eliminatioon
A = np.array([
    [2, 3, -1],
    [4, 4, -3],
    [2, -3, 1]])

b = np.array([5, 3, -1])

solution = gauss_elimination(A, b, use_pivoting=True)
print(solution)

In [ ]:
# LU Decomposition
# A = np.array([
#     [3, 2, 4],
#     [0, 0.333, 0.666],
#     [0, 0, -8]])

# b = np.array([1, 1.666, 0])

# A = np.array([
#     [1, 0, 0],
#     [0.333, 1, 0],
#     [1.333, 1, 1]])

# b = np.array([1, 2, 3])

A = np.array([
    [1, 5, 5], 
    [6, 9, 22], 
    [32, 5., 5]])

b = np.array([1, 2, 7])

solution, LU, Pivot, PdU, Info = LU_decomposition(A, b)

print("Solution:", solution)
print("LU matrix:\n", LU)
print("Pivot indices:", Pivot)
print("Product of the diagonal of U:", PdU)
print("Info:", Info) 